In [81]:
import pandas as pd
import os
import re

files = [*(os.scandir('csvs'))]
csv_list = []
reg = r'(\d*)_data.csv'

data_dict = {}
for f in files:
    station = re.findall(reg, f.name)[0]
    data_dict[station] = pd.read_csv(f'csvs/{f.name}',
                                     index_col=0)
    # data_dict[station] = data_dict[station].loc[data_dict[station]['대여량'] > 0]
data_dict[station]

,요일,am/pm,시간대,대여량,강수량,기온,풍속
0,0,0,0,0,0.4,27.2,1.6
1,0,0,1,0,0.4,27.2,1.6
2,0,0,2,0,0.4,27.2,1.6
3,0,0,3,0,0.4,27.2,1.6
4,0,0,4,0,0.4,27.2,1.6
...,...,...,...,...,...,...,...
3835,5,1,7,1,0.0,-1.1,0.9
3836,5,1,8,1,0.0,-1.1,0.9
3837,5,1,9,1,0.0,-1.1,0.9
3838,5,1,10,0,0.0,-1.1,0.9


In [82]:
from sklearn.preprocessing import OneHotEncoder

encoded_dfs = {}
for station, df in data_dict.items():
    # 원 핫 인코더
    encoder = OneHotEncoder(sparse_output=False)
    # 요일, 시간대를 인코딩.
    encoded_weekday = encoder.fit_transform(df[['요일']])
    encoded_time = encoder.fit_transform(df[['시간대']])
    temp = df.copy()
    # 원래 데이터에서 요일, 시간대 열을 제거
    temp = temp.drop(columns=['요일', '시간대'])
    # 인코딩된 데이터를 데이터프레임으로 변환
    enc_df_weekday = pd.DataFrame(encoded_weekday, columns=[f'요일_{i}' for i in range(7)])
    enc_df_time = pd.DataFrame(encoded_time, columns=[f'시간대_{i}' for i in range(12)])
    # 최종 결과물 삽입
    encoded_dfs[station] = pd.concat([temp, enc_df_weekday, enc_df_time], axis=1)
# test_target = '191'
# encoder = OneHotEncoder(sparse=False)
# target = data_dict[test_target]
# encoded_data = encoder.fit_transform(target[['요일']])
# encoded_data
# copy = target.copy()
# copy = copy.drop(columns=['요일'])
# encoded_df = pd.DataFrame(encoded_data, columns=['요일_' + str(i) for i in range(7)])
# copy = pd.concat([copy, encoded_df], axis=1)
# copy
encoded_dfs['194'].columns

Index(['am/pm', '대여량', '강수량', '기온', '풍속', '요일_0', '요일_1', '요일_2', '요일_3',
       '요일_4', '요일_5', '요일_6', '시간대_0', '시간대_1', '시간대_2', '시간대_3', '시간대_4',
       '시간대_5', '시간대_6', '시간대_7', '시간대_8', '시간대_9', '시간대_10', '시간대_11'],
      dtype='object')

In [83]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# models = {}
# for k, v in encoded_dfs.items():
#     model = LinearRegression()
#     y = v['대여량']
#     x = v.drop(columns=['대여량'])
#     model.fit(x, y)
#     models[k] = model
# models
# models = {}
# for k, v in encoded_dfs.items():
#     model = RandomForestRegressor(n_estimators=100)
#     y = v['대여량']
#     x = v.drop(columns=['대여량'])
#     model.fit(x, y)
#     models[k] = model
# models
models = {}
for k, v in encoded_dfs.items():
    model = XGBRegressor(eta=0.3)
    y = v['대여량']
    x = v.drop(columns=['대여량'])
    model.fit(x, y)
    models[k] = model
models

{'194': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.3, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...),
 '768': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.3, eval_me

In [84]:
import joblib
for k, v in models.items():
    joblib.dump(v, f'models/{k}_xb.pkl')